In [ ]:
import os
from tqdm.notebook import tqdm

import cv2

import PIL
from PIL import Image
from scipy import ndimage

import math
import numpy as np

import torch
import torch.nn as nn

import numba

import matplotlib.pyplot as plt

In [ ]:
TestSet1_folder = 'data/TestSet1'
TestSet2_folder = 'data/TestSet2'
TestSet3_folder = 'data/TestSet3'

TestSet1_result_folder = 'result/TestSet1'
TestSet2_result_folder = 'result/TestSet2'
TestSet3_result_folder = 'result/TestSet3'

os.makedirs(TestSet1_result_folder, exist_ok=True)
os.makedirs(TestSet2_result_folder, exist_ok=True)
os.makedirs(TestSet3_result_folder, exist_ok=True)

In [ ]:
TestSet1_imgs = []
for file in os.listdir(TestSet1_folder): 
    TestSet1_imgs.append(
        cv2.cvtColor(np.asarray(Image.open(os.path.join(TestSet1_folder, file))),
                     cv2.COLOR_BGR2GRAY)
    )

In [ ]:
TestSet2_imgs = []
for file in os.listdir(TestSet2_folder): 
    TestSet2_imgs.append(
        cv2.cvtColor(np.asarray(Image.open(os.path.join(TestSet2_folder, file))),
                     cv2.COLOR_BGR2GRAY)
    )

In [ ]:
TestSet3_imgs = []
for file in os.listdir(TestSet3_folder): 
    TestSet3_imgs.append(
        cv2.cvtColor(np.asarray(Image.open(os.path.join(TestSet3_folder, file))),
                     cv2.COLOR_BGR2GRAY)
    )

In [ ]:
# adaptive_thresholding:
blockSize = 39
C = 16

In [ ]:
def is_square(anchor):
    x, y, w, h = cv2.boundingRect(anchor)
    ratio = float(w)/h
    return ratio >= 0.9 and ratio <= 1.1

In [ ]:
def find_qr_codes_anchors(img):
    # бинаризируем изображение
    thresh_img = cv2.adaptiveThreshold(img, 255, 
            cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, blockSize=blockSize, C=C)
    # ищем контуры
    cnts, hierarchy = cv2.findContours(thresh_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    anchors = []
    for i, cnt in enumerate(cnts):
        # аппроксимирем многоугольником, ищем квадоты
        cnt = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), closed=True)
        if hierarchy[0][i][2] > 0 and hierarchy[0][i][3] > 0 and hierarchy[0][hierarchy[0][i][2]][2] > 0:
            cnt = cv2.approxPolyDP(cnt, 10, closed=True)
            if cnt.shape[0] == 4 and is_square(cnt):
                anchors.append(cnt)
    return (img, anchors)

In [ ]:
for i, image in tqdm(list(enumerate(TestSet1_imgs))):
    img, anchors = find_qr_codes_anchors(image)
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(img, cmap='gray')
    for anchor in anchors:
        plt.plot(list(anchor[:, :, 0].flatten()) + list(anchor[:, :, 0][0]), 
                 list(anchor[:, :, 1].flatten()) + list(anchor[:, :, 1][0]))
    plt.savefig(os.path.join(TestSet1_result_folder, f'img_{i}.png'))
    plt.show()

In [ ]:
for i, image in tqdm(list(enumerate(TestSet2_imgs))):
    img, anchors = find_qr_codes_anchors(image)    
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(img, cmap='gray')
    for anchor in anchors:
        plt.plot(list(anchor[:, :, 0].flatten()) + list(anchor[:, :, 0][0]), 
                 list(anchor[:, :, 1].flatten()) + list(anchor[:, :, 1][0]))
    plt.savefig(os.path.join(TestSet2_result_folder, f'img_{i}.png'))
    plt.show()

Описание алгоритма: <br>
    Адаптивно локально бинаризируем изображение
    С помощью opencv находим контуры фигур и иерархию контуров <br>
    Апроксимируем найденные контуры более простыми кривыми и ищем кривые приближенно похожие на квадраты <br>
    Ищем с помощью найденных контуров и иерархии "опорные" точки qr-кодов
    

На один прогон изображений из двух TestSet'ов размером 116 картинок тратится ~ 1 минута 56 секунд. Итого время работы алгоритма ~1.22 секунды на изображение.


Для TestSet1 имеем 63 правильных срабатываний и 3 неправильных, всего опорных точек на изображениях 144, т.е. **precision** ~95.5% и **recall** ~43.8% <br>
Для TestSet2 множества имеем 106 правильных срабатываний и 6 неправильное, всего опорных точек на изображениях 147, т.е. **precision** ~94.6% и **recall** ~72.2% <br>

В целом можно воспользоватся функцией cv::QRCodeDetector::detect, чтобы найти сам QR-код, и уже внутри него искать паттерны. Но это, кажется, немного нечестным...

### TestSet3

In [ ]:
for i, image in tqdm(list(enumerate(TestSet3_imgs))):
    img, anchors = find_qr_codes_anchors(image)
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(img, cmap='gray')
    for anchor in anchors:
        plt.plot(list(anchor[:, :, 0].flatten()) + list(anchor[:, :, 0][0]), 
                 list(anchor[:, :, 1].flatten()) + list(anchor[:, :, 1][0]))
    plt.savefig(os.path.join(TestSet3_result_folder, f'img_{i}.png'))
    plt.show()

Для TestSet3 множества имеем 356 правильных срабатываний и 70 неправильное, всего опорных точек на изображениях 483, т.е. precision ~83.6% и recall ~73.7%